In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import table


In [2]:
IRAF_SENSFUNC = '/Users/afloers/Downloads/sens_FORS2_P104_300I.dat'
PERIOD = 'P104'
GRISM = '300I'

In [18]:
iraf = np.loadtxt(IRAF_SENSFUNC)
wave_iraf = iraf[:,0]
sens_iraf = iraf[:,1]

iraf_bin = wave_iraf[1]-wave_iraf[0]
extend_by = 200
lower_extension = np.linspace(wave_iraf[0] - 2 * extend_by * iraf_bin, wave_iraf[0] - iraf_bin, 2 * extend_by)
upper_extension = np.linspace(wave_iraf[-1] + iraf_bin,wave_iraf[-1] + 4 * extend_by * iraf_bin, 4 * extend_by)

z1 = np.polyfit(wave_iraf[:30], sens_iraf[:30], 1)
z2 = np.polyfit(wave_iraf[-30:], sens_iraf[-30:], 1)

lower_sens_extension = z1[1] + z1[0]*lower_extension 
lower_sens_extension[lower_sens_extension<0] = np.nan

upper_sens_extension = z2[1] + z2[0]*upper_extension 
upper_sens_extension[upper_sens_extension<0] = np.nan

wave_iraf_extended = np.hstack((lower_extension, wave_iraf, upper_extension))
sens_iraf_extended = np.hstack((lower_sens_extension, sens_iraf, upper_sens_extension))

turn_off_point = 5950
value_turn_off = np.interp(turn_off_point, wave_iraf_extended, sens_iraf_extended)
sens_iraf_extended[wave_iraf_extended < turn_off_point] = value_turn_off

turn_off_point_red = 10300
value_turn_off_red = np.interp(turn_off_point_red, wave_iraf_extended, sens_iraf_extended)
sens_iraf_extended[wave_iraf_extended > turn_off_point_red] = value_turn_off_red


sens_iraf_extended = sens_iraf_extended + 2.5 * np.log10(iraf_bin)

sens_iraf_extended = (10**(0.4 * sens_iraf_extended))
sens = fits.open('data/FORSify_sens_template.fits').copy()
header = sens[0].header
sens['WAVE'].data = wave_iraf_extended
sens['SENSFUNC'].data = 1/(sens_iraf_extended*1e-17)
meta_table = table.Table(sens['METADATA'].data)
header['SPC1DFIL'] = 'ESO FORS2 standards' + PERIOD
sens[0].header = header
out_table = table.Table(meta={'name': 'Sensitivity Function'})
out_table['WAVE'] = [wave_iraf_extended.T]
out_table['SENSFUNC'] = [(1/(sens_iraf_extended*1e-17)).T]
out_table['MASK_SENS'] = [np.zeros(sens_iraf_extended.shape[0], dtype=bool).T]
out_table['WAVE_MIN'] = wave_iraf_extended.min()
out_table['WAVE_MAX'] = wave_iraf_extended.max()


hdu_out_table = fits.table_to_hdu(out_table)
hdu_out_table.name = 'OUT_TABLE'
sens['OUT_TABLE'] = hdu_out_table
meta_table['EXPTIME'] = 'NaN'
meta_table['AIRMASS'] = 'NaN'
meta_table['STD_RA'] = 'NaN'
meta_table['STD_DEC'] = 'NaN'
meta_table['STD_NAME'] = 'NaN'
meta_table['CAL_FILE'] = PERIOD + ' Standards'
hdu_meta_table = fits.table_to_hdu(meta_table)
hdu_meta_table.name = 'METADATA'
sens['METADATA'] = hdu_meta_table
sens.writeto('pypeit_sens_'+ PERIOD + '_' + GRISM + '.fits', overwrite=True, checksum=True)

In [15]:
%matplotlib
import matplotlib.pyplot as plt
plt.plot(wave_iraf_extended, sens_iraf_extended)

Using matplotlib backend: TkAgg
